In [7]:
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Auto select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------------------------------
# Load metadata
# ---------------------------------------------------
with open("/kaggle/input/cub2002011/CUB_200_2011/train_test_split.txt") as f:
    split = dict(line.strip().split() for line in f)

with open("/kaggle/input/cub2002011/CUB_200_2011/images.txt") as f:
    paths = dict(line.strip().split() for line in f)

with open("/kaggle/input/cub2002011/CUB_200_2011/image_class_labels.txt") as f:
    labels = dict(line.strip().split() for line in f)

# ---------------------------------------------------
# Select only FIRST 5 classes
# ---------------------------------------------------
selected_classes = set(list({int(v) for v in labels.values()})[:200])

print("Using classes:", selected_classes)

train_paths, train_labels = [], []
test_paths, test_labels = [], []

base = "/kaggle/input/cub2002011/CUB_200_2011/images/"

for img_id, rel in paths.items():
    cls = int(labels[img_id])
    if cls not in selected_classes:
        continue

    full = base + rel
    if split[img_id] == "1":
        train_paths.append(full)
        train_labels.append(cls)
    else:
        test_paths.append(full)
        test_labels.append(cls)

print("Train images:", len(train_paths))
print("Test images :", len(test_paths))

# ---------------------------------------------------
# Convert to DataFrames (path + class)
# ---------------------------------------------------
train_df = pd.DataFrame({"path": train_paths, "class": train_labels})
test_df  = pd.DataFrame({"path": test_paths , "class": test_labels})

# ---------------------------------------------------
# Dataset class with transforms
# ---------------------------------------------------
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

class CUBDataset(Dataset):
    def __init__(self, df, transform):
        self.paths = df["path"].values
        self.labels = df["class"].values
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        img = self.transform(img)
        cls = self.labels[idx] - 1
        return img, cls

# ---------------------------------------------------
# Dataloaders
# ---------------------------------------------------
train_dataset = CUBDataset(train_df, transform_train)
test_dataset = CUBDataset(test_df, transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset , batch_size=32, shuffle=False, num_workers=2)

print("Train loader batches:", len(train_loader))
print("Test loader batches :", len(test_loader))


Using device: cuda
Using classes: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200}
Train images: 5994
Test images : 5794
Train loader batches: 188
Test load

In [8]:
# without DVML

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------------------------
# Encoder + Decoder
# -------------------------------------------------------
class SimpleMetricAutoEncoder(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        goog = models.googlenet(weights="IMAGENET1K_V1")

        self.backbone = nn.Sequential(
            goog.conv1, goog.maxpool1,
            goog.conv2, goog.conv3, goog.maxpool2,
            goog.inception3a, goog.inception3b, goog.maxpool3,
            goog.inception4a, goog.inception4b, goog.inception4c,
            goog.inception4d, goog.inception4e,
            goog.maxpool4,
            goog.inception5a, goog.inception5b,
            goog.avgpool
        )

        self.fc_embed = nn.Linear(1024, embed_dim)

        self.decoder = nn.Sequential(
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024)
        )

    def extract_f(self, x):
        f = self.backbone(x)
        return f.view(f.size(0), -1)

    def forward(self, x):
        f = self.extract_f(x)
        z = self.fc_embed(f)
        f_hat = self.decoder(z)
        return z, f, f_hat


# -------------------------------------------------------
# N-Pair Loss
# -------------------------------------------------------
class NPairLoss(nn.Module):
    def forward(self, A, P):
        A = F.normalize(A, dim=1)
        P = F.normalize(P, dim=1)
        sim = A @ P.t()
        lbl = torch.arange(len(A), device=sim.device)
        return F.cross_entropy(sim, lbl)

def recon_loss(f, f_hat):
    return F.mse_loss(f_hat, f)


# -------------------------------------------------------
# Initialize model
# -------------------------------------------------------
model = SimpleMetricAutoEncoder(embed_dim=128).to(device)

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
npair = NPairLoss()

# -------------------------------------------------------
# TRAIN LOOP
# -------------------------------------------------------
for epoch in range(3):
    model.train()
    for imgs, labels in train_loader:

        imgs, labels = imgs.to(device), torch.tensor(labels).to(device)

        z, f, f_hat = model(imgs)

        A, P = [], []
        for c in labels.unique():
            idx = (labels == c).nonzero().flatten()
            if len(idx) >= 2:
                A.append(z[idx[0]])
                P.append(z[idx[1]])

        if len(A) < 1:
            continue

        A = torch.stack(A)
        P = torch.stack(P)

        loss = npair(A, P) + recon_loss(f, f_hat)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} Loss = {loss.item():.4f}")


# -------------------------------------------------------
# EXTRACT EMBEDDINGS
# -------------------------------------------------------
def extract_embeddings(model, loader):
    E, L = [], []
    model.eval()
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(device)
            z, _, _ = model(imgs)
            E.append(z.cpu())
            L.append(torch.tensor(lbls))
    return torch.cat(E), torch.cat(L)


# -------------------------------------------------------
# RECALL@K
# -------------------------------------------------------
def recall_at_k(E, L, K=1):
    E = F.normalize(E, dim=1)
    S = E @ E.t()
    N = len(L)
    S[range(N), range(N)] = -1
    _, idx = S.topk(K, dim=1)
    correct = sum(L[i] in L[idx[i]] for i in range(N))
    return correct / N


# -------------------------------------------------------
# EVALUATE
# -------------------------------------------------------
E, L = extract_embeddings(model, test_loader)
for k in [1,2,4]:
    print(f"R@{k} =", recall_at_k(E, L, K=k))

Using 2 GPUs


/tmp/ipykernel_48/1033272441.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs, labels = imgs.to(device), torch.tensor(labels).to(device)


Epoch 0 Loss = 0.2993
Epoch 1 Loss = 0.9533
Epoch 2 Loss = 0.0093


/tmp/ipykernel_48/1033272441.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L.append(torch.tensor(lbls))


R@1 = 0.2002071108042803
R@2 = 0.28926475664480494
R@4 = 0.3962720055229548


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DVML(nn.Module):
    def __init__(self, embed_dim=128, var_dim=128):
        super().__init__()
        goog = models.googlenet(weights="IMAGENET1K_V1")
        self.backbone = nn.Sequential(
            goog.conv1, goog.maxpool1,
            goog.conv2, goog.conv3, goog.maxpool2,
            goog.inception3a, goog.inception3b, goog.maxpool3,
            goog.inception4a, goog.inception4b, goog.inception4c,
            goog.inception4d, goog.inception4e,
            goog.maxpool4,
            goog.inception5a, goog.inception5b,
            goog.avgpool
        )
        self.fc_mu = nn.Linear(1024, embed_dim)
        self.fc_logvar = nn.Linear(1024, var_dim)
        self.decoder = nn.Sequential(
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024)
        )

    def encode(self, x):
        f = self.backbone(x)
        f = f.view(f.size(0), -1)
        mu = self.fc_mu(f)
        logvar = self.fc_logvar(f)
        return mu, logvar, f

    def reparam(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std, eps * std

    def forward(self, x):
        mu, logvar, f = self.encode(x)
        z, noise = self.reparam(mu, logvar)
        f_hat = self.decoder(z)
        return z, mu, logvar, f, f_hat, noise


class NPairLoss(nn.Module):
    def forward(self, A, P):
        A = F.normalize(A, dim=1)
        P = F.normalize(P, dim=1)
        sim = A @ P.t()
        lbl = torch.arange(len(A), device=sim.device)
        return F.cross_entropy(sim, lbl)


def kl_loss(mu, logvar):
    return -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())


def recon_loss(f, f_hat):
    return F.mse_loss(f_hat, f)


model = DVML(embed_dim=128, var_dim=128).to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
npair = NPairLoss()


def build_pairs(z, labels):
    A, P = [], []
    for c in labels.unique():
        idx = (labels == c).nonzero().flatten()
        if len(idx) >= 2:
            A.append(z[idx[0]])
            P.append(z[idx[1]])
    if len(A) == 0:
        return None, None
    return torch.stack(A), torch.stack(P)


def build_synthetic_pairs(mu, noise, labels):
    z_syn = mu + noise
    return build_pairs(z_syn, labels)


for epoch in range(20):
    model.train()
    kl_w = min(1.0, epoch / 10)
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        z, mu, logvar, f, f_hat, noise = model(imgs)

        A_real, P_real = build_pairs(z, labels)
        if A_real is None:
            continue

        A_syn, P_syn = build_synthetic_pairs(mu, noise, labels)
        if A_syn is None:
            continue

        L_real = npair(A_real, P_real)
        L_syn = npair(A_syn, P_syn)
        L_recon = recon_loss(f, f_hat)
        L_kl = kl_loss(mu, logvar)

        loss = L_real + L_syn + L_recon + kl_w * L_kl

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(epoch, loss.item())


def extract_embeddings(model, loader):
    E, L = [], []
    model.eval()
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(device)
            z, _, _, _, _, _ = model(imgs)
            E.append(z.cpu())
            L.append(lbls.cpu())
    return torch.cat(E), torch.cat(L)


def recall_at_k(E, L, K=1):
    E = F.normalize(E, dim=1)
    S = E @ E.t()
    N = len(L)
    S[range(N), range(N)] = -1
    _, idx = S.topk(K, dim=1)
    return sum(L[i] in L[idx[i]] for i in range(N)) / N


E, L = extract_embeddings(model, test_loader)
for k in [1,2,4,8]:
    print("R@", k, "=", recall_at_k(E, L, k))

0 0.0262776967138052
